Variable

In [10]:
import copy

X=Y=5
DEPTH = 4
REUSSITE = 4
ALGO_SYMB="O"
morpion=[[' ' for _ in range(X)] for _ in range(Y)]


Vérifie la Victoire

In [3]:
def check_victoire(grille, symbole, longueur):
    directions = [(1, 0), (0, 1), (1, 1), (1, -1)]  # droite, bas, diag-bas, diag-haut

    for y in range(Y):
        for x in range(X):
            if grille[y][x] != symbole:
                continue
            for dx, dy in directions:
                count = 1
                nx, ny = x + dx, y + dy
                while 0 <= nx < X and 0 <= ny < Y and grille[ny][nx] == symbole:
                    count += 1
                    if count >= longueur:
                        return True
                    nx += dx
                    ny += dy
    return False


Vérifie si une victoire est possible en un coup

In [4]:
def potvic(grille, symbole, reussite,X,Y):
    directions = [(-1,-1),(0,-1),(1,-1),(1,0),(1,1),(0,1),(-1,1),(-1,0)]
    l=[]
    for y in range(Y):
        for x in range(X):
            if grille[y][x] == ' ':
                coor=(x,y)
    
                vic=[0 for i in range(8)]
                place=0

                for dx, dy in directions:
                    count=0
                    nx, ny = coor[0] + dx, coor[1] + dy
                    while 0 <= nx < X and 0 <= ny < Y and grille[ny][nx] == symbole:
                        count += 1
                        nx += dx
                        ny += dy
                    vic[place]=count
                    place+=1  

                for i in range(8):
                    if vic[i]!=0 and 0<= i+4 <7 and vic[i+4] != 0 and vic[i]+vic[i+4]+1 >=reussite:
                        l.append(coor)
                    elif vic[i]+1>=reussite: 
                        l.append(coor)
    return l

Compte le nombre de case vide en liste

In [5]:
def casevide(grille,X,Y):
    l=[]
    for y in range(Y):
        for x in range(X):
            if grille[y][x] == ' ':
                l.append((x,y))
    return l

Calcule du nombre de point

In [6]:
def calcpoint(grille, symbole,reussite,X,Y):
    point=0
    if check_victoire(grille,symbole,reussite):
        point+=50
        
    point+=len(potvic(grille,symbole,reussite,X,Y))*10
    if symbole == "X":
        symbole="O"
    else:
        symbole="X"
        
    if check_victoire(grille,symbole,reussite):
        point-=50
    return point

Crée le dico d'enfent remplie eu meme de dico et d'enfant ... Je crée l'abre

createarbre(Le morpion,dernier endroit joué,symbole qui a joué,reussite,X,Y,couche actuelle,couche Max,symbole du robot)

In [7]:
def createarbre(grille,coor,symbole,reussite,X,Y,depth,maxdepth,basesymbole):
    fut_child=casevide(grille,X,Y)

    if depth == maxdepth or check_victoire(grille,symbole,reussite) or len(fut_child) == 0:
        return {"coor":coor,"child":[],"score":calcpoint(grille,basesymbole,reussite,X,Y),"depth":depth,"symbole" : symbole}
    
    else:        
        children = []
        for i in fut_child:
            newgrille=copy.deepcopy(grille)
            newgrille[i[1]][i[0]]="O" if symbole == "X" else "X"
            children.append(createarbre(newgrille,i,"O" if symbole == "X" else "X",reussite,X,Y,depth+1,maxdepth,basesymbole))
        return {"coor":coor,"child":children,"score":None,"depth":depth,"symbole" : symbole}


MinMax fonction

In [8]:
def minmax(node,symbole):
    if node["score"] == None:
        if node["symbole"] == symbole:
            node["score"]=min([minmax(i,symbole) for i in node["child"]])
            return node["score"]
        else: 
            node["score"]=max([minmax(i,symbole) for i in node["child"]])
            return node["score"]
    else:
        return node["score"]

def ultimatefinalresultatofoneweekwork(grille,coor,symbole,reussite,X,Y,depth,basesymbole):
    node=createarbre(grille,coor,symbole,reussite,X,Y,0,depth,basesymbole)
    minmax(node,basesymbole)
    maxi=node["child"][0]
    for i in node["child"]:
        if i["score"] > maxi["score"]:
            maxi=i
    return maxi["coor"]

#ultimatefinalresultatofoneweekwork(morpion,(0,0),"X",REUSSITE,X,Y,DEPTH,ALGO_SYMB)

En pygame maintenant

In [11]:
import pygame

N = 100
screen = pygame.display.set_mode((N * X, N * Y))
running = True
Play="X" if ALGO_SYMB == "O" else "O"

def dess(grille):
    for y in range(Y):
        for x in range(X):
            if grille[y][x] == "X":
                pygame.draw.rect(screen, (255, 0, 0), pygame.Rect(x * N, y * N, N - 1, N - 1))
            elif grille[y][x] == "O":
                pygame.draw.rect(screen, (0, 255, 0), pygame.Rect(x * N, y * N, N - 1, N - 1))
            else:
                pygame.draw.rect(screen, (0, 0, 0), pygame.Rect(x * N, y * N, N - 1, N - 1))

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            pos = pygame.mouse.get_pos()
            pos=( pos[0] // N, pos[1] // N )
            if morpion[pos[1]][pos[0]] == ' ':
                morpion[pos[1]][pos[0]] = Play
                if check_victoire(morpion, Play, REUSSITE):
                    print("Victoire de", Play)
                    running = False
                    continue
                elif casevide(morpion,X,Y) == []:
                    print("Match nul")
                    running = False
                    continue
            if Play == "X":
                new_pos=ultimatefinalresultatofoneweekwork(morpion,pos,"X",REUSSITE,X,Y,DEPTH,ALGO_SYMB)
                morpion[new_pos[1]][new_pos[0]] = ALGO_SYMB
                if check_victoire(morpion, ALGO_SYMB, REUSSITE):
                    print("Victoire de", ALGO_SYMB)
                    running = False
                    continue
                elif casevide(morpion,X,Y) == []:
                    print("Match nul")
                    running = False
                    continue
    dess(morpion)
    pygame.display.update()
pygame.quit()


Match nul
